<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e4be70f1fa25dbbfa11635332b34448de1be8019e666854d7acca76f18570ece
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-12 09:53:18
-------------------
qualified stocks: 84
with latest results: 24
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  5.59 L
Current:  1.44 C
-------------------
Today PnL: 96.02 K (0.67%)
Current PnL: -21.31 L (-13.96%)
CY Booked + Current PnL: -7.82 L (-5.12%)
-------------------
Total profit:  1.75 L
Total loss:  -23.06 L
-------------------
Total Booked + Current PnL: 19.16 L (15.25%)
Total Booked PnL: 40.47 L (32.22%)
Curr Year Booked PnL: 13.49 L (9.36%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.30 C
Est FTT PnL: 85.97 L (59.65%)
Deployed:  1.26 C
Current:  1.44 C
CAGR/XIRR %: 7.99%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.00,101.49,51.0,M-SC,2.54,86788.0,-13989.0,14086.0,0.13,-13.88,16.23,0.09,245.0,-0.99,0.63,14.02,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.65,45.0,X-MC,3.30,88185.0,-2493.0,17399.0,-0.52,-2.75,19.73,16.44,101.0,-0.14,0.64,16.87,X40,ATH,PHARMA
41,ITC,452.00,-38.34,46.0,X-LC,0.82,199072.0,-1066.0,21958.0,0.06,-0.53,11.03,10.44,4.0,-0.05,1.44,5.00,X40,NTT,FMCG
33,ICICIGI,2252.93,-13.14,57.0,X-MC,7.55,187507.0,13187.0,22013.0,1.01,7.56,11.74,20.19,91.0,0.60,1.35,23.82,X40,ATH,INSURANCE
37,INDIGOPNTS,1408.00,126.56,77.0,M-SC,4.52,148558.0,-26001.0,26027.0,-0.37,-14.90,17.52,0.02,221.0,-1.00,1.07,29.22,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,CAMS,4762.00,-1.62,60.0,X-SC,1.24,263868.0,12012.0,50425.0,4.16,4.77,19.11,24.79,122.0,0.24,1.90,29.01,X40N,NTT,MISC
59,ROUTE,2227.21,-57.12,36.0,H-SC,23.24,70362.0,-62704.0,154585.0,3.67,-47.12,219.70,69.05,118.0,-0.41,0.51,3.67,SR,ATH,IT
8,ATULAUTO,844.00,3694.59,57.0,M-SC,4.07,170340.0,-26792.0,116615.0,2.73,-13.59,68.46,45.57,236.0,-0.23,1.23,20.77,XY24,NTT,AUTO
5,ANGELONE,3033.00,25.51,72.0,X-SC,6.92,222040.0,31034.0,26667.0,2.51,16.25,12.01,30.21,157.0,1.16,1.60,40.52,X40N,NTT,FINANCE
75,TRIDENT,48.00,-8.57,55.0,M-SC,2.09,71111.0,-21144.0,47445.0,2.35,-22.92,66.72,28.51,224.0,-0.45,0.51,21.68,XR,NTT,TEXTILES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VBL,671.64,-17.78,49.0,X-LC,5.67,296000.0,-19842.0,132519.0,-1.41,-6.28,44.77,35.67,5.0,-0.15,2.14,6.70,X40N,ATH,FMCG
17,CERA,9475.00,-27.21,33.0,H-SC,2.02,132825.0,-43078.0,85101.0,-0.76,-24.49,64.07,23.89,149.0,-0.51,0.96,14.71,OX40N,NTT,CERAMICS
72,TATAMOTORS,1065.00,-53.17,24.0,X-LC,37.92,147959.0,-128564.0,238643.0,-0.69,-46.49,161.29,39.81,54.0,-0.54,1.07,4.29,XY24,NTT,AUTO
67,STARHEALTH,761.00,18.81,54.0,H-SC,14.47,264738.0,-16385.0,149259.0,-0.68,-5.83,56.38,47.26,144.0,-0.11,1.91,41.84,XY24,NTT,INSURANCE
69,SYMPHONY,1306.00,-44.94,42.0,M-SC,15.79,117356.0,-53785.0,53726.0,-0.67,-31.43,45.78,-0.03,196.0,-1.00,0.85,2.43,OX40N,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RECLTD,446.0,42.86,47.0,M-LC,4.28,201850.0,-1320.0,43458.0,1.37,-0.65,21.53,20.74,55.0,-0.03,1.46,6.17,XY25,NTT,FINANCE
83,WIPRO,420.0,-13.79,53.0,M-LC,5.74,151516.0,571.0,108879.0,1.11,0.38,71.86,72.51,53.0,0.01,1.09,6.42,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,101.49,51.0,M-SC,2.54,86788.0,-13989.0,14086.0,0.13,-13.88,16.23,0.09,245.0,-0.99,0.63,14.02,OX40N,NTT,DURABLES
37,INDIGOPNTS,1408.0,126.56,77.0,M-SC,4.52,148558.0,-26001.0,26027.0,-0.37,-14.90,17.52,0.02,221.0,-1.00,1.07,29.22,OX40N,NTT,PAINTS
45,KANSAINER,340.0,-68.34,38.0,H-SC,2.27,218475.0,-51192.0,87521.0,-0.45,-18.98,40.06,13.47,138.0,-0.58,1.58,11.09,XY24,NTT,PAINTS
71,TATAELXSI,9161.0,-24.72,47.0,H-MC,6.39,101888.0,-29947.0,72167.0,1.50,-22.72,70.83,32.03,98.0,-0.41,0.74,14.12,OX40N,NTT,IT
65,SIS,528.0,2024.35,48.0,H-SC,2.21,85731.0,-25301.0,48910.0,0.66,-22.79,57.05,21.26,156.0,-0.52,0.62,15.83,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,WIPRO,420.00,-13.79,53.0,M-LC,5.74,151516.0,571.0,108879.0,1.11,0.38,71.86,72.51,53.0,0.01,1.09,6.42,XR,NTT,IT
36,INDIAMART,4810.62,-52.77,59.0,H-SC,4.89,131869.0,8533.0,123100.0,1.47,6.92,93.35,106.72,119.0,0.07,0.95,31.91,AR,ATH,MISC
60,SAMMAANCAP,326.00,-193.18,56.0,M-SC,35.67,160956.0,11736.0,132451.0,1.41,7.86,82.29,96.62,208.0,0.09,1.16,74.19,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,INDIAMART,4810.62,-52.77,59.0,H-SC,4.89,131869.0,8533.0,123100.0,1.47,6.92,93.35,106.72,119.0,0.07,0.95,31.91,AR,ATH,MISC
83,WIPRO,420.00,-13.79,53.0,M-LC,5.74,151516.0,571.0,108879.0,1.11,0.38,71.86,72.51,53.0,0.01,1.09,6.42,XR,NTT,IT
82,WHIRLPOOL,2270.00,-45.21,37.0,M-SC,8.85,113546.0,-1252.0,95288.0,-0.30,-1.09,83.92,81.92,223.0,-0.01,0.82,34.42,XR,NTT,DURABLES
46,KPIGREEN,731.05,3.65,46.0,H-SC,10.21,123581.0,-1716.0,60641.0,0.84,-1.37,49.07,47.03,141.0,-0.03,0.89,54.73,MH,ATH,POWER
7,ASIANTILES,137.00,7638.89,69.0,L-SC,15.05,84891.0,-8919.0,85536.0,1.10,-9.51,100.76,81.67,269.0,-0.10,0.61,64.39,XR,NTT,CERAMICS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-53.17,24.0,X-LC,37.92,147959.0,-128564.0,238643.0,-0.69,-46.49,161.29,39.81,54.0,-0.54,1.07,4.29,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86959.0,-41711.0,87011.0,0.08,-32.42,100.06,35.20,219.0,-0.48,0.63,0.08,X40,NTT,FOOTWEAR
25,GILLETTE,11206.78,-17.47,27.0,X-MC,5.73,253800.0,-96.0,82409.0,2.12,-0.04,32.47,32.42,105.0,-0.00,1.83,15.06,X40,ATH,FMCG
53,QUESS,986.00,-51.52,31.0,X-SC,37.15,47878.0,-17128.0,168057.0,0.06,-26.35,351.01,232.18,198.0,-0.10,0.35,0.06,XY24,NTT,MISC
44,JSWINFRA,342.00,-24.16,32.0,X-MC,4.71,190519.0,-9828.0,40333.0,-0.41,-4.91,21.17,15.23,178.0,-0.24,1.38,25.76,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,ITC,452.00,-38.34,46.0,X-LC,0.82,199072.0,-1066.0,21958.0,0.06,-0.53,11.03,10.44,4.0,-0.05,1.44,5.00,X40,NTT,FMCG
52,PGHH,17907.65,-32.00,39.0,X-MC,1.09,196980.0,-3840.0,71642.0,1.66,-1.91,36.37,33.76,80.0,-0.05,1.42,3.19,X40,ATH,FMCG
64,SIEMENS,4671.50,-5.39,40.0,H-LC,1.67,151955.0,-34140.0,81615.0,0.57,-18.35,53.71,25.51,15.0,-0.42,1.10,11.98,AR,ATH,ELECTRICAL
29,HAVELLS,2069.16,-8.73,39.0,X-MC,3.31,304584.0,-23407.0,129936.0,0.47,-7.14,42.66,32.48,92.0,-0.18,2.20,3.47,X40,ATH,ELECTRICAL
77,UNITDSPR,1644.00,-6.66,61.0,X-LC,3.46,244618.0,10474.0,38160.0,0.54,4.47,15.60,20.77,86.0,0.27,1.77,11.70,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-51.52,31.0,X-SC,37.15,47878.0,-17128.0,168057.0,0.06,-26.35,351.01,232.18,198.0,-0.10,0.35,0.06,XY24,NTT,MISC
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86959.0,-41711.0,87011.0,0.08,-32.42,100.06,35.20,219.0,-0.48,0.63,0.08,X40,NTT,FOOTWEAR
1,ABB,7934.00,-42.78,33.0,H-LC,5.00,239304.0,-22315.0,141524.0,0.14,-8.53,59.14,45.57,7.0,-0.16,1.73,1.83,AR,NTT,ELECTRICAL
10,BAJAJHFL,181.50,-15.96,38.0,X-MC,6.32,177153.0,-23353.0,120499.0,0.32,-11.65,68.02,48.45,90.0,-0.19,1.28,2.16,X40N,ATH,FINANCE
20,COLPAL,3726.84,-6.30,44.0,X-MC,7.01,218600.0,-44765.0,154091.0,0.18,-17.00,70.49,41.51,84.0,-0.29,1.58,2.58,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-51.52,31.0,X-SC,37.15,47878.0,-17128.0,168057.0,0.06,-26.35,351.01,232.18,198.0,-0.10,0.35,0.06,XY24,NTT,MISC
57,RELAXO,1176.00,-40.65,42.0,X-SC,3.72,80962.0,-64198.0,140129.0,-0.29,-44.23,173.08,52.31,136.0,-0.46,0.58,8.37,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86959.0,-41711.0,87011.0,0.08,-32.42,100.06,35.20,219.0,-0.48,0.63,0.08,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.65,45.0,X-MC,3.30,88185.0,-2493.0,17399.0,-0.52,-2.75,19.73,16.44,101.0,-0.14,0.64,16.87,X40,ATH,PHARMA
32,HONAUT,58357.33,-24.23,37.0,X-SC,0.93,106755.0,-21183.0,68313.0,0.23,-16.56,63.99,36.84,143.0,-0.31,0.77,9.29,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-27.08,60.0,X-LC,8.99,291127.0,-54829.0,121516.0,1.64,-15.85,41.74,19.28,1.0,-0.45,2.10,7.63,X40,ATH,IT
39,INFY,2275.00,-15.29,67.0,X-LC,4.55,329639.0,16751.0,154930.0,1.13,5.35,47.00,54.87,3.0,0.11,2.38,14.08,X40,BTT,IT
41,ITC,452.00,-38.34,46.0,X-LC,0.82,199072.0,-1066.0,21958.0,0.06,-0.53,11.03,10.44,4.0,-0.05,1.44,5.00,X40,NTT,FMCG
80,VBL,671.64,-17.78,49.0,X-LC,5.67,296000.0,-19842.0,132519.0,-1.41,-6.28,44.77,35.67,5.0,-0.15,2.14,6.70,X40N,ATH,FMCG
1,ABB,7934.00,-42.78,33.0,H-LC,5.00,239304.0,-22315.0,141524.0,0.14,-8.53,59.14,45.57,7.0,-0.16,1.73,1.83,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,45.0,L-SC,26.27,77741.0,-35808.0,75860.0,1.48,-31.54,97.58,35.27,268.0,-0.47,0.56,90.27,XR,NTT,HOTELS
7,ASIANTILES,137.00,7638.89,69.0,L-SC,15.05,84891.0,-8919.0,85536.0,1.10,-9.51,100.76,81.67,269.0,-0.10,0.61,64.39,XR,NTT,CERAMICS
49,MASFIN,398.61,-20.05,47.0,H-SC,7.86,90975.0,-7005.0,28612.0,0.26,-7.15,31.45,22.05,152.0,-0.24,0.66,32.24,XR,ATH,FINANCE
46,KPIGREEN,731.05,3.65,46.0,H-SC,10.21,123581.0,-1716.0,60641.0,0.84,-1.37,49.07,47.03,141.0,-0.03,0.89,54.73,MH,ATH,POWER
36,INDIAMART,4810.62,-52.77,59.0,H-SC,4.89,131869.0,8533.0,123100.0,1.47,6.92,93.35,106.72,119.0,0.07,0.95,31.91,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7638.89,69.0,L-SC,15.05,84891.0,-8919.0,85536.0,1.10,-9.51,100.76,81.67,269.0,-0.10,0.61,64.39,XR,NTT,CERAMICS
78,VAIBHAVGBL,521.00,69.70,59.0,H-SC,6.56,147204.0,-35571.0,146645.0,1.34,-19.46,99.62,60.77,125.0,-0.24,1.06,32.57,XR,NTT,JEWELLERY
5,ANGELONE,3033.00,25.51,72.0,X-SC,6.92,222040.0,31034.0,26667.0,2.51,16.25,12.01,30.21,157.0,1.16,1.60,40.52,X40N,NTT,FINANCE
37,INDIGOPNTS,1408.00,126.56,77.0,M-SC,4.52,148558.0,-26001.0,26027.0,-0.37,-14.90,17.52,0.02,221.0,-1.00,1.07,29.22,OX40N,NTT,PAINTS
6,ASIANPAINT,3465.66,-7.44,74.0,X-LC,6.81,233737.0,-18031.0,71243.0,-0.04,-7.16,30.48,21.13,27.0,-0.25,1.69,25.73,X40,ATH,PAINTS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.51
1,25,45.99
2,50,77.59


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.06
LC    29.95
MC    25.02
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.98
X40      19.98
X40N     12.89
XY25     10.54
XR       10.03
AR        7.96
OX40N     6.81
X200      1.89
SR        1.06
MH        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.69
X-MC    22.24
X-LC    21.65
M-SC    13.02
X-SC     5.83
H-LC     4.72
M-LC     2.55
H-MC     2.43
L-SC     1.52
L-LC     1.03
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.47,-4.02,37.66
IT,13.18,-15.17,76.82
FINANCE,12.58,-8.51,57.70
MISC,7.99,-17.37,76.57
PAINTS,5.94,-12.33,29.07
ELECTRICAL,5.80,-12.59,52.50
INSURANCE,4.59,0.61,34.42
AUTO,2.92,-42.12,101.43
BANKS,2.82,-37.19,126.61


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3121458.0,20
XR,1266140.0,13
AR,1184539.0,9
X40,925295.0,13
X40N,683211.0,9
OX40N,544247.0,9
XY25,475236.0,7
SR,273405.0,2
X200,62555.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3409358.0,24
X-MC,1328595.0,15
M-SC,1241826.0,15
X-LC,1043541.0,12
X-SC,540602.0,6
H-LC,285694.0,3
H-MC,252224.0,2
L-SC,250837.0,3
M-LC,152337.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1179917.0      6
           AR         837937.0      5
           XR         798503.0      7
M-SC       XY24       719495.0      6
X-MC       XY24       565383.0      4
X-LC       X40        422161.0      5
X-MC       X40        416123.0      7
X-LC       XY24       308549.0      2
X-SC       X40N       285534.0      4
H-SC       SR         273405.0      2
           OX40N      258955.0      3
X-LC       X40N       236845.0      3
H-LC       AR         223139.0      2
X-MC       XY25       186257.0      2
H-MC       XY24       180057.0      1
X-SC       XY24       168057.0      1
L-SC       XR         161396.0      2
X-MC       X40N       160832.0      2
M-SC       XR         142733.0      2
           XY25       132451.0      1
           OX40N      123684.0      4
           AR         123463.0      2
M-LC       XR         108879.0      1
L-SC       OX40N       89441.0      1
X-SC       X40         87011.0      1
X-LC       XY25        75986.0      2
H-MC       OX40N       72167.0      1
H-LC       X200        62555.0      1
H-SC       MH          60641.0      1
L-MC       XR          54629.0      1
M-LC       XY25        43458.0      1
L-LC       XY25        37084.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
